In [1]:
import librosa
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import librosa.display
import IPython.display as ipd
import os
from tqdm import tqdm
import warnings
import pickle
from audio2numpy import open_audio
from nnAudio import Spectrogram
import torch
from sys import getsizeof
import random

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [3]:
data_dir = '/Users/madhavapaliyam/Documents/CMSC/rando/raganet/data'
hop_length = 1024

image_length = 800

def save_chunks(ragam, start_from = 0):
    ragam_dir = os.path.join(data_dir, ragam)
    cqt_list = np.zeros((0,84,image_length))
    failed = []
    for i,song in enumerate(listdir_nohidden(ragam_dir)):
        if i > start_from or i == 322:
            path = os.path.join(ragam_dir,song)
            print(path)
            try:
                signal, sampling_rate = open_audio(path)
            except:
                failed.append(song)
                continue

            if len(signal.shape) > 1: 
                signal = np.mean(signal, axis = 1)

            # extract harmonic elements (data_h) bc we dont need mridangam :(
            data_h, data_p = librosa.effects.hpss(signal)
            # constant q transform
            S = np.real(librosa.cqt(data_h, sr=sampling_rate, hop_length=hop_length)).astype(np.float32)
            # trim first and last 20 ish seconds 
            S = S[:, 700:-700]

            if S.shape[1] >= image_length: 
                num_groups = S.shape[1] // image_length
                bound = num_groups * image_length
                S = S[:,0:bound]
                chunks = np.split(S, num_groups, axis = 1)
                cqt_list = np.concatenate((cqt_list, np.array(chunks)))

            print("Finished song:", song, "Shape so far:", str(cqt_list.nbytes), "Number of Chunks: ", len(chunks))
            if cqt_list.shape[0] > 4000:
                np.save(str(ragam) + "_" + str(i) + ".npy", cqt_list)
                print("SAVED at" + str(i))
                cqt_list = np.zeros((0,84,image_length))
        else:
            continue 
    np.save(str(ragam) + "_" + str(i) + ".npy", cqt_list)
    print("SAVED at" + str(i))
    print("FAILED:", failed)
    return cqt_list

In [4]:
%%time
chr_list = save_chunks('AbhEri', 317)

/Users/madhavapaliyam/Documents/CMSC/rando/raganet/data/AbhEri/06-bhajarE_rE_mAnasa_shrI-AbhEri.mp3


KeyboardInterrupt: 

In [ ]:


s_list = save_chunks('shuddhadhanyAsi')

In [ ]:
np.save("shuddadhanyasi.npy", s_list)

In [ ]:
s_list.shape

In [ ]:
chr_list.shape

In [ ]:
plt.figure(figsize=(15, 5))
librosa.display.specshow(s_list[4,:,0:20].astype(float), x_axis='time', hop_length=hop_length)

In [ ]:
plt.figure(figsize=(15, 5))
librosa.display.specshow(s_list[4,:,0:20], x_axis='time', hop_length=hop_length)

In [ ]:
s = np.load("../../data/npy/shuddadhanyasi.npy")

In [ ]:
np.zeros((4426, 84, 800)).astype(np.float64).nbytes

In [ ]:
r = 'shuddadhanyasi.npy'
r[:-3]

In [ ]:
np.tile(1,6)

In [ ]:
q = []
q.extend(s)
q.extend(s)

In [ ]:
np.array(q).shape